Трекинг и подсчет людей:

In [1]:
import cv2
import random
import numpy as np
from ultralytics import YOLO

def process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="output_video_with_counter.mp4"):
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")

    # Get input video frame rate and dimensions
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the output video writer
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Define color for bounding boxes
    box_color = (0, 255, 0)  # Green color

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform tracking and detection
        results = model.track(frame, iou=0.4, conf=0.5, persist=True, imgsz=640, verbose=False, tracker="botsort.yaml")

        # Initialize object count
        people_count = 0

        if results[0].boxes.id is not None:  # this will ensure that id is not None -> exist tracks
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id in zip(boxes, ids):
                # Increment object count
                people_count += 1

                # Draw bounding box and ID
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), box_color, 2)
                cv2.putText(
                    frame,
                    f"Id {id}",
                    (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 255),
                    2,
                )

        # Display the object count on the frame
        cv2.putText(frame, f"People: {people_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        if save_video:
            out.write(frame)

        if show_video:
            frame = cv2.resize(frame, (0, 0), fx=0.75, fy=0.75)
            cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release the input video capture and output video writer
    cap.release()
    if save_video:
        out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

Использвание фунции

In [2]:
if __name__ == "__main__":
    model = YOLO('best.pt')
    input_video_path = 'test.mp4'
    process_video_with_tracking(model, input_video_path, show_video=True, save_video=True)

Воспроизведение записи

In [1]:
import os

# Путь к вашему видео
video_path = 'output_video_with_counter.mp4'

# Открываем видео с помощью системного приложения
os.system(f'start {video_path}')

0